# Train model in SFT fashion

In this notebook, there are different aproaches for Supervised fine-tuning

## Read Libraries

In [1]:
! pip install transformers datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.4 MB/s eta 0:00:00


In [2]:
! pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.5 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelWithLMHead,GPT2Model,TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import pandas as pd
from datasets import Dataset
from peft import PeftModel, PeftConfig
import random

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Prepare the data

The data needed to be loaded ftom Google drive and the bineary sentiment needed to be transformed to the description (positive and negative). Also for testing two data sets have been created. One for training and another one for validation.

In [5]:
 path="/content/drive/MyDrive/Masterthesis/Data/data_processed/train_for_sft.tsv"
 data=pd.read_csv(path, sep='\t')
 data=data[["preprocessed_text","sentiment"]]
 data

,preprocessed_text,sentiment
0,Es gibt so viele verschiedene Freizeitaktivitä...,1
1,Das Hotel kann man nicht weiter empfehlen. Das...,0
2,"Regisseur John Polson bemüht sich, Spannung au...",0
3,Absolut nicht zu empfehlen!!! Die Lage des Hau...,0
4,"Die Lage ist der Hammer. Zum Strand, der natür...",1
...,...,...
73601,Ganz einfach: Super! Ich habe selten in einem ...,1
73602,"Animation: Es gibt zur Zeit nur Animateure, di...",0
73603,"Super Hotel, werden Wiederholungstäter Die Hot...",1
73604,Katalog und Realität stimmen nicht überein Hot...,0


In [6]:
def sentiment_map(x):
  if x==1:
    return "positive"
  if x==0:
    return "negative"

In [7]:
data["sentiment"]=data["sentiment"].apply(lambda x:sentiment_map(x) )



In [8]:
 path_val="/content/drive/MyDrive/Masterthesis/Data/data_processed/val_for_sft.tsv"
 data_val=pd.read_csv(path_val, sep='\t')
 data_val=data_val[["preprocessed_text","sentiment"]]


In [9]:
data_val["sentiment"]=data_val["sentiment"].apply(lambda x:sentiment_map(x) )


###Second Approach - these models were selected for RL

Here the sentiment is created while training the model on the new instruction column, that has been created as the combination of the sentiment token and the sentence. This approach generates sentiment based sentences.


Selected for thesis.

In [10]:
def create_instruction(sent,text):
  '''
  create instructions that will be used for supervised fien-tuning.
  As an instruction we create text in the form "[sentiment] text".
  Finally we take first 512 words as the instruction.
  '''

  text=f"[{sent}] {text}"
  words = text.split()
  text =' '. join(words[:512])
  return text

In [11]:
data["instructions"]=data.apply(lambda row: create_instruction(row['sentiment'], row['preprocessed_text']), axis=1)
data["instructions"]

0        [positive] Es gibt so viele verschiedene Freiz...
1        [negative] Das Hotel kann man nicht weiter emp...
2        [negative] Regisseur John Polson bemüht sich, ...
3        [negative] Absolut nicht zu empfehlen!!! Die L...
4        [positive] Die Lage ist der Hammer. Zum Strand...
                               ...                        
73601    [positive] Ganz einfach: Super! Ich habe selte...
73602    [negative] Animation: Es gibt zur Zeit nur Ani...
73603    [positive] Super Hotel, werden Wiederholungstä...
73604    [negative] Katalog und Realität stimmen nicht ...
73605    [negative] Unfreundlich, Qualität und Angebot ...
Name: instructions, Length: 73606, dtype: object

In [12]:
dataset = Dataset.from_pandas(data)#create dataset from pandas dataset
dataset

Dataset({
    features: ['preprocessed_text', 'sentiment', 'instructions'],
    num_rows: 73606
})

create tokenizer and model:

In [13]:
tokenizer = AutoTokenizer.from_pretrained("benjamin/gpt2-wechsel-german")
model = AutoModelWithLMHead.from_pretrained("benjamin/gpt2-wechsel-german")

tokenizer_config.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/856k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/514k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/665M [00:00<?, ?B/s]

In [14]:
tokenizer.pad_token = tokenizer.eos_token


Add training configs. Here we train the model for 1,2 and 3 epochs.

In [15]:
config=TrainingArguments(
    output_dir="test",
    learning_rate=5e-5,
    num_train_epochs=3###change to 1,2,3

)

In [16]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="instructions",
    max_seq_length=1024,
    args=config
)

Map:   0%|          | 0/73606 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


In [ ]:
trainer.train()


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.174100
1000,3.171300
1500,3.193600
2000,3.202200
2500,3.203600
3000,3.231500
3500,3.256000
4000,3.252300
4500,3.260600
5000,3.285300


TrainOutput(global_step=27603, training_loss=3.1648802103410834, metrics={'train_runtime': 4252.0965, 'train_samples_per_second': 51.932, 'train_steps_per_second': 6.492, 'total_flos': 3.0273064155648e+16, 'train_loss': 3.1648802103410834, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")

Here the model is tested. As an example we take a sentiment token and the beginning of sentence.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_3_epochs")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
prompt_sent="[negative]: Obwohl ich mir Mühe gegeben habe,"

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 3,
    "top_p": 0.5,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 100,
    "eos_token_id": -1,
}

In [ ]:
generation_kwargs = {
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 56,
    "num_beams":4,
    "no_repeat_ngram_size":3,
    "num_return_sequences":1
}

In [ ]:
input_ids = tokenizer.encode(prompt_sent, return_tensors="pt")

In [ ]:
output = model.generate(   input_ids,
  **generation_kwargs)

generated_text = tokenizer.decode(output[0])
print("Generated Text:")
print(generated_text)

Generated Text:
[negative]: Obwohl ich mir Mühe gegeben habe, war das Essen nicht gut. Es gab jeden Tag das gleiche. Die Auswahl war sehr gering, es gab nur eine Sorte Fleisch und eine Sorte Gemüse. Zudem gab es jeden Tag Nudeln mit Tomatensoße. Das Frühstück war ok, aber das Abendessen war auch nicht besser. Jeden Tag
